In [1287]:
#IMPORT
import random
from sage.modules.free_module import Module
import hashlib
from itertools import chain
import hashlib
import binascii
import sys

In [1288]:
#PARAMS
p = 31
Fp = GF(p)
n = 256
k = 52
r = n-k
t = n
z = 3
Fz = GF(z)
lamb = 128
lev_leaf = log(t,2)
Fp_set = Set(Fp)
Fp_star = Fp_set.difference([0])
msg = "buonasera"

In [1289]:
#FUNZIONI DI UTILITA

def generate_random_seed():
    set_random_seed()
    return initial_seed()

def set_seed(seed):
    set_random_seed(seed)
    return initial_seed()

def hash_and_get_seed(input):
    hash = hashlib.sha256(str(input).encode()).digest()
    seed = int.from_bytes(hash, 'big')
    return seed

def hexhash(input):
    return hashlib.sha256(input.encode()).hexdigest()

def convert_int_to_hash_string(input):
    return input.to_bytes((input.bit_length() + 7) // 8, 'big').hex()

def expand_seed(seed_init):
    seed_str = str(seed_init)
    seeds = hashlib.sha256(seed_str.encode()).digest()
    
    seed_1 = seeds[:16] #i primi 16 byte
    seed_2 = seeds[16:] #gli ultimi 16 byte
    
    seed_1 = int.from_bytes(seed_1, 'big')
    
    seed_2 = int.from_bytes(seed_2, 'big')
    
    return seed_1, seed_2
#t=8
# #w = 5

#0,2,4,5,7

#Creo Generatore g del Campo E
def get_generator(p, z, Fp):
    assert is_prime(p),"p is not prime!"
    assert (p-1)%z == 0, "z does not divide p-1!"
    alpha = Fp.primitive_element()
    #non ricordo perchè exp e g sono scelti secondo questa logica
    exp = (p-1)/z
    g = alpha**exp
    return g

def create_random_vector(domain, n_elems, seed):
    set_seed(seed)
    return random_vector(domain, n_elems)

def create_random_matrix(domain, n_rows, n_columns, seed):
    set_seed(seed)
    M = random_matrix(domain, n_rows, n_columns)
    return M

#Creo vettore errore con elementi in E e lunghezza V 
def create_vEn(g, n, Fz, Fp, seed):
    eta = create_random_vector(Fz, n, seed)
    e = vector(Fp, n)
    for i in range(0, n):#range check
        e[i] = (g**eta[i])
    return e

def create_vector_over_Fp_star(Fp_star, t, seed):
    set_random_seed(seed)
    return [Fp_star.random_element() for i in range(t)]

def componentwise_multiply(v1, v2):
    if len(v1) != len(v2):
        raise ValueError("I vettori devono avere la stessa dimensione")
    
    result = vector(GF(p), len(v1))  # Crea un nuovo vettore per i risultati
    for i in range(len(v1)):
        result[i] = v1[i] * v2[i]  # Moltiplica gli elementi corrispondenti
    
    return result

In [1290]:
#SEED TREE
class SeedTree:
    def __init__(self, depth, salt, seed_m):
        if(depth>0):
            self.seed_m = seed_m
            self.depth = depth
            self.salt = str(salt)
            self.tree = None
            self.full = False
            self.seed_path = None
        else:
            raise Exception('Il numero delle foglie deve essere una porenza di 2')

    #METODI PUBBLICI 
    def make_tree(self):
        if self.full:
            raise Exception('Albero gia creato')
        else:
            #BISOGNA AGGIUNGERE CONTROLLI?
            self.tree = []
            self.__make_tree(self.seed_m, 0)
            self.full = True
        
    def get_leaves(self):
        if self.full:
            return self.tree[self.depth]
        else:
            raise Exception('Albero non creato')

    def get_seed_path(self, list_nodes):
        if self.full:
            self.seed_path={}
            self.__get_seed_path(self.depth-1, list_nodes)
            return self.seed_path
        else:
            raise Exception('Albero non creato o foglia non valida')

#da finire
    def verify_proof(self, node, list_proof):
        if self.full:  
            list_proof.append(node)
            return self.__verify_proof(list_proof)
        else:
            raise Exception('Albero non creato')  
    
    #METODI PRIVATI

    def __make_tree(self, list_nodes, depth):
        if(depth<=self.depth):
            list_children = self.__create_new_level(list_nodes, depth)
            self.tree.append(list_children)
            self.__make_tree(list_children, depth+1)

    def __create_new_level(self, parents, level):
        children = []
        if level == 0:
            sx, dx = self.__compute_children_value(parents, '')
            children.append(sx)
            children.append(dx)
        else:
            base = 2**level
            for i in range(0, len(parents)):
                sx, dx = self.__compute_children_value(parents[i], base+i)
                children.append(sx)
                children.append(dx)
        return children
        
    def __compute_children_value(self, seed_parent, index_parent):
        nodes = str(seed_parent) + self.salt + str(index_parent)
        sx, dx = expand_seed(int(nodes))  
        return sx, dx
    
    #provvisorio
    def __get_seed_path(self, depth, list_index):
        if len(list_index)>0 and depth>0:
            list_parent = sorted(list(set(map(lambda elem: ((elem // 2) + 1) if (elem // 2) % 2 == 0 else (elem // 2) - 1, list_index))))#prendo gli indici dei nodi padri eliminando i duplicati
            list_sub_root = list(filter(lambda x: ((x % 2 == 0 and x + 1 not in list_parent) or (x % 2 != 0 and x - 1 not in list_parent)), list_parent))
            list_new_children = list(filter(lambda x:  x not in list_sub_root, list_parent))
            self.seed_path[depth] = dict(map(lambda x: (x, self.tree[depth][x]), list_new_children))
            self.__get_seed_path(depth-1, list_new_children)
        

    #da finire
    def __verify_proof(self, list_proof, index):
        while len(list_proof) > 1:
            ultimo = list_proof.pop()
            penultimo = list_proof.pop()
            to_hash= penultimo +ultimo
            hash=hashlib.sha256(to_hash.encode()).hexdigest()
            list_proof.append(hash)
        return list_proof[0] == self.get_root()
 

In [1291]:
#MERKLE TREE

class MerkleTree:
    def __init__(self, n_leaves):
        if(n_leaves%2==0):
            self.n_leaves = n_leaves
            self.depth = log(n_leaves, 2)
            self.tree =  None
            self.merkle_proof =  None
            self.full = False
        else:
            raise Exception('Il numero delle foglie deve essere una porenza di 2')

    #METODI PUBBLICI 
    def make_tree(self, leaves):
        if isinstance(leaves, list) and len(leaves) == self.n_leaves and not(self.full):
            self.tree = []
            self.tree.append(leaves)
            self.__make_tree(leaves, self.depth-1)
            self.full = True
        else:
            raise Exception('Il numero delle foglie deve essere pari a {}'.format(self.n_leaves))
       
    def get_root(self):
        if self.full:
            return self.tree[0][0]
        else:
            raise Exception('Albero non creato')

    def get_proof(self, list_nodes):
        if all(self.full or 1 <= node <= 2**self.depth for node in list_nodes):
            list_leaves = list(map(lambda x: x+1 if x % 2 == 0 else x-1, list_nodes))
            list_leaves = list(filter(lambda x: x not in list_nodes, list_leaves))
            list_indexes = self.__get_indexes(self.depth, list_leaves, [])
            self.__get_merkle_proof(list_indexes)
            return self.merkle_proof
        else:
            raise Exception('Albero non creato o foglia non valida')

    #VEDREMO
    def verify_proof(self, node, list_proof):
        if self.full:  
            list_proof.append(node)
            return self.__verify_proof(list_proof)
        else:
            raise Exception('Albero non creato')  
    
    #METODI PRIVATI

    def __make_tree(self, list_nodes, depth):
        if(depth>=0):
            list_parent = self.__create_new_level(list_nodes)
            self.tree.insert(0, list_parent)
            self.__make_tree(list_parent, depth-1)

    def __create_new_level(self, child_nodes):
        parents = []
        for i in range(0, len(child_nodes), 2):
            new_node = self.__compute_father_value(child_nodes[i], child_nodes[i+1])
            parents.append(new_node)
        return parents
        
    def __compute_father_value(self, sx, dx):
        return hexhash(sx+dx)
    
    def __get_indexes(self, depth, list_index, list_proof):
        if(depth > 0):
            list_proof.insert(0, list_index)
            list_parent = sorted(list(set(map(lambda elem: ((elem // 2) + 1) if (elem // 2) % 2 == 0 else (elem // 2) - 1, list_index))))
            return self.__get_indexes(depth-1, list_parent, list_proof)
        else:
            return list_proof


    def __get_merkle_proof(self, list_merkle_proofs):
        self.merkle_proof = {}    
        for i in range(len(list_merkle_proofs)): 
            current_proof = list_merkle_proofs[i] #controllo livello i esimo
            if i < len(list_merkle_proofs) - 1:
                next_proof = list_merkle_proofs[i+1]
                current_proof = list(filter(lambda x: x*2 not in next_proof and (x*2)+1 not in next_proof, current_proof))
            if not len(current_proof)==0:
                self.merkle_proof[i+1] = dict(map(lambda x: (x, self.tree[i+1][x]), current_proof))

    #da finire
    def __verify_proof(self, list_proof, index):
        while len(list_proof) > 1:
            ultimo = list_proof.pop()
            penultimo = list_proof.pop()
            to_hash= penultimo +ultimo
            hash=hashlib.sha256(to_hash.encode()).hexdigest()
            list_proof.append(hash)
        return list_proof[0] == self.get_root()

In [1292]:
#GENERAZIONE CHIAVI
def key_gen():
    seed_sk = generate_random_seed() 
    seed_e , seed_pk = expand_seed(seed_sk)
    V = create_random_matrix(Fp, k, r, seed_pk) 
    g = get_generator(p, z, Fp)
    e = create_vEn(g, n, Fz, Fp, seed_e)
    s = e[0:r] + e[r:n]*V
    return seed_sk, seed_pk, s


In [1293]:
#FIRMA
def key_material_expansion(seed_sk):
    seed_e , seed_pk = expand_seed(seed_sk)
    V = create_random_matrix(Fp, k, r, seed_pk)
    eta = create_random_vector(Fz, n, seed_e)
    return V, eta

def compute_commitment(V, eta):
    seed_m = generate_random_seed() 
    seed_salt = generate_random_seed()
    salt = hash_and_get_seed(seed_salt)
    seed_tree = SeedTree(lev_leaf, salt, seed_m)
    seed_tree.make_tree()
    seeds = seed_tree.get_leaves()
    cmt0 = [] 
    cmt1 = []
    for i in range(0, t):
        seed_ui, seed_ei = expand_seed(seeds[i])
        eta_i = create_random_vector(Fz, n, seed_ei)
        sigma_i = eta-eta_i #sigma_i ∈ Fz^ns
        v = vector(Fp,n) #v∈ Fp^n
        g = get_generator(p, z, Fp)
        for j in range(0, n-1):
            v[j] = (g**sigma_i[j])  
        ui = create_random_vector(Fp, n, seed_ui)
        u = componentwise_multiply(v, ui)
        st = u[0:r] + u[r:n]*V
        commit0 = hexhash( str(st) + str(sigma_i) + str(salt) + str(i) )
        commit1 = hexhash( str(seeds[i]) + str(salt) + str(i) )
        cmt0.append(commit0) 
        cmt1.append(commit1)
    merkle_tree = MerkleTree(t)
    merkle_tree.make_tree(cmt0)
    c0 = merkle_tree.get_root()
    input_c1 = "".join(map(str, cmt1))
    c1 = hexhash(input_c1)
    return seed_m, seeds, salt, c0, c1, merkle_tree, seed_tree

def generate_first_challenge(c0, c1, msg, salt):
    salt = convert_int_to_hash_string(salt)
    input_ch1 = c0+c1+msg+salt
    seed_ch1 = hash_and_get_seed(input_ch1)
    ch1 = create_vector_over_Fp_star(Fp_star, t, seed_ch1)
    return ch1

def generate_first_response(seeds, ch1):
    list_hash_yi=[]
    for seed_i, ch_i_1 in zip(seeds, ch1):
        seed_ui, seed_ei = expand_seed(seed_i)
        g = get_generator(p, z, Fp)
        ei = create_vEn(g, n, Fz, Fp, seed_ei)
        ui = create_random_vector(Fp, n, seed_ui)
        yi = ui+(ch_i_1)*ei
        hi = hexhash(str(yi))
        list_hash_yi.append(hi)
    h = hexhash("".join(list_hash_yi))
    return ch1, h

def generate_second_challenge(c0, c1, msg, salt, ch1, h):#, seed_tree):
    salt = convert_int_to_hash_string(salt)
    input_ch2 = c0+c1+"".join(map(str, ch1))+h+msg+salt
    seed_ch2 = hash_and_get_seed(input_ch2)
    ch2 = create_random_vector(GF(2), t, seed_ch2)
    return ch2

def generate_second_response(ch2, merkle_tree, seed_tree):
    index_1, index_0 = [], []
    [index_1.append(i) if x == 1 else index_0.append(i) for i, x in enumerate(ch2)]
    merkle_proofs = merkle_tree.get_proof(index_0)
    seed_path = seed_tree.get_seed_path(index_1)
    return merkle_proofs, seed_path


In [1294]:
seed_sk, seed_pk, s = key_gen()
V, eta = key_material_expansion(seed_sk)
seed_m, seeds, salt, c0, c1, merkle_tree, seed_tree = compute_commitment(V, eta)
ch1 = generate_first_challenge(c0, c1, msg, salt)
ch1, h = generate_first_response(seeds, ch1)
ch2 = generate_second_challenge(c0, c1, msg, salt, ch1, h)
merkle_proofs, seed_path = generate_second_response(ch2, merkle_tree, seed_tree)

In [1295]:
print(merkle_proofs)


{6: {1: 'ddbd296dfb3f87714b519e1cf598f428d3fad2fb10c8bf7c739fe9a14b4b6af2', 6: '597d133add0896903d2a43c0200c7bf1356da4edab08a223d7bd62349c8ced75', 14: 'c2e83b38f1c187ae18ed0f99d7b4e71212accf2cb19e83098f72b521bd2e9dcb', 18: 'be4e7ab9aa2b8c0553e62eb24929b8b5f464d47aa654415e17fe4760b9e0ed2f', 23: '87cca0aa1022773c03d1008683408e9f0ed776465a819fd4e2dcf0229b014003', 26: '7183d533f1f8796397050626ddf5d21fdb25a974772a6e0edf4a18d65a74e06e', 33: '8f9d9785d69780ee2c0a0bd9f8ce1976ae23b2ab73f78732f002ca0ce0e30b3a', 36: '4e97cd54a2b2db1b8591b6e8a3a0b45cf10dca53a2478d272df88d04883a6353', 38: 'ac0e1d35f018d933ec88c86dc42e732f744d68a4f64433c4bdf1691684fe3f53', 43: 'df84e2fb3bd26c79e00ae121b78bdbea2cb122cf4eb3b9b9ba2461a704fa1604', 45: '7e9d84a87d3d8c4d4f65b5084d49e40ae4c758ff7a8105859cf2bd2e1e8ce0c8', 47: '3d0bfc7816df3242fc0f018d8c048577d3c237e04d89b48793c33ed63fea383c', 54: '73d6ea882c9185c87e896484610626890688393a570dc86e8017c0874ed6c322', 56: 'c0257141c8ce8cccbc3d89821ac59046d21c44e6648dd9de05b58f8c

In [1296]:
print(seed_path)

{7: {0: 152054995861607741432481830482883266025, 1: 250062195120342061142059609963934125849, 6: 318755054288223216730274134531131265188, 7: 252826997722784147915153263742516742147, 8: 236859811017459994767737891201950564265, 9: 79203319188424091541982867654605373384, 10: 290018386877190417569652182183738691184, 11: 4564686719790332961260162232873242908, 16: 50540118137512296279670958110937275991, 17: 123196411840051075631886956951693600605, 18: 228562390488432156180209574300874757286, 19: 196597764477297452772837087492265732411, 24: 272915289029160159017647951542848655744, 25: 228649678674479867843968477454025452655, 26: 39492014146543973419486282445083454500, 27: 198524624249248115397028494875488825548, 28: 227827429810247933980809150582750758796, 29: 110151171720878833973135355766168026867, 30: 67660489579233206530613794948444522039, 31: 300485691199597695533713472148475899182, 32: 39183359749174127800720151236215826285, 33: 271321742002242097030562298059705020457, 34: 25937899391455